# Pre-processing techniques

In [11]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
df = pd.read_csv('../../data/final_features_df.csv')
df.head()

,Unnamed: 0,Age,Income,faves_pca0,faves_pca1,unfaves_pca0,unfaves_pca1,accessories,alcohol,animamted,...,Drama.2,Entertainment (Variety Shows),Factual,Learning,Music,News,Religion &amp; Ethics,Sport.1,Weather,Rating_bin
0,0,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
1,1,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
2,2,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
3,3,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,4,62,1,-0.321485,0.0786,-0.19967,-0.200645,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0


In [7]:
df_0 = df.fillna(0)

In [8]:
df = df_0.sample(frac = 1)
train_frac = 0.8
test_frac = 0.1

X_train = df[[c for c in df.columns if c != 'Rating_bin']].iloc[:int(len(df) * train_frac)].values
y_train = df.Rating_bin.iloc[:int(len(df) * train_frac)].values

X_test = df[[c for c in df.columns if c != 'Rating_bin']].iloc[int(len(df) * train_frac):int(len(df) * (train_frac+test_frac))].values
y_test = df.Rating_bin.iloc[int(len(df) * train_frac):int(len(df) * (train_frac+test_frac))].values

X_valid = df[[c for c in df.columns if c != 'Rating_bin']].iloc[int(len(df) * (train_frac+test_frac)):].values
y_valid = df.Rating_bin.iloc[int(len(df) * (train_frac+test_frac)):].values

In [10]:
pd.DataFrame(X_train).to_csv('../../data/X_train.csv')
pd.DataFrame(y_train).to_csv('../../data/y_train.csv')

pd.DataFrame(X_test).to_csv('../../data/X_test.csv')
pd.DataFrame(y_test).to_csv('../../data/y_test.csv')

pd.DataFrame(X_valid).to_csv('../../data/X_valid.csv')
pd.DataFrame(y_valid).to_csv('../../data/y_valid.csv')

## Baseline model: DecisionTree

In [12]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3146
           1       0.41      0.38      0.39       466

    accuracy                           0.85      3612
   macro avg       0.66      0.65      0.65      3612
weighted avg       0.84      0.85      0.85      3612



array([[2886,  260],
       [ 288,  178]])

## Reweight